# Bagian Baru

AZKAA RAHIILA HARDI
2022071052
SISTEM CERDAS

In [127]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [128]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense

In [132]:
#database
u_cols = ['user_id', 'age', 'sex', 'occupation', 'Zip_Code',]
users = pd.read_csv('/content/gdrive/MyDrive/sisdas/ml-100k/u.user', sep='|', names = u_cols, encoding='latin-1')

movies_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'Action',
               'animation', 'Drama', 'Comedy', 'musical', 'animal', 'kids',
               'mystery', 'sci-fi', 'war', 'fantasy', 'horror', 'bollywood' , 'western']
movies = pd.read_csv('/content/gdrive/MyDrive/sisdas/ml-100k/u.item', sep='|', names = movies_cols, encoding='latin-1')

#ratings
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/gdrive/MyDrive/sisdas/ml-100k/ua.base', sep='\t', names=ratings_cols, encoding='latin-1')


In [133]:
users.head()

,user_id,age,sex,occupation,Zip_Code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [134]:
movies.head()

,,,,movie_id,title,release_date,video_release_date,imdb_url,unknown,Action,animation,Drama,Comedy,musical,animal,kids,mystery,sci-fi,war,fantasy,horror,bollywood,western
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [124]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [125]:
# Pisahkan data menjadi data latih (training data) dan data uji (test data)
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

# Fungsi untuk mendapatkan film yang belum pernah diberi rating oleh pengguna
def movies_not_rated_by_user(user_id):
    user_rated_movies = set(train_data[train_data['user_id'] == user_id]['movie_id'])
    all_movies = set(np.arange(movies.shape[0]))
    not_rated_movies = list(all_movies - user_rated_movies)
    return not_rated_movies

# Fungsi untuk mendapatkan judul film berdasarkan ID film
def get_movie_titles(movie_ids):
    return movies[movies['movie_id'].isin(movie_ids)]['title']

# ...

# Create model
# Membangun model NCF menggunakan TensorFlow/Keras
def create_ncf_model(num_users, num_items, latent_dim=8):
    # Input layer untuk user
    user_input = Input(shape=(1,))
    user_embedding = Embedding(num_users + 1, latent_dim)(user_input)  # Tambahkan +1 di sini
    user_flat = Flatten()(user_embedding)

    # Input layer untuk item (film)
    item_input = Input(shape=(1,))
    item_embedding = Embedding(num_items + 1, latent_dim)(item_input)  # Tambahkan +1 di sini
    item_flat = Flatten()(item_embedding)

    # Concatenate user dan item embeddings
    concat = Concatenate()([user_flat, item_flat])

    # Fully connected layer
    hidden = Dense(64, activation='relu')(concat)
    output = Dense(1)(hidden)

    # Compile model
    model = Model(inputs=[user_input, item_input], outputs=output)
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

# Train model
# Melatih model menggunakan data latih
model = create_ncf_model(users['user_id'].nunique(), movies['movie_id'].nunique())
model.fit([train_data['user_id'], train_data['movie_id']], train_data['rating'], batch_size=64, epochs=10,
          validation_data=([test_data['user_id'], test_data['movie_id']], test_data['rating']))

# ...
# Evaluate model
test_loss = model.evaluate([test_data['user_id'], test_data['movie_id']], test_data['rating'])
print(f"Test Loss: {test_loss}")

# Make recommendations
user_id = 123
user_movies = movies_not_rated_by_user(user_id)
user_input = np.array([user_id] * len(user_movies))
user_movies_input = np.array(user_movies).reshape(-1, 1)

user_input = user_input.reshape((-1, 1))
user_movies_input = user_movies_input.reshape((-1, 1))

predictions = model.predict([user_input, user_movies_input]).flatten()

top_movie_indices = np.argsort(predictions)[::-1][:10]
recommended_movie_ids = [user_movies[i] for i in top_movie_indices]
recommended_movies = get_movie_titles(recommended_movie_ids)

print("Recommended movies for user", user_id, ":", recommended_movies.values)

Epoch 1/10
1133/1133 [==============================] - 13s 9ms/step - loss: 1.8580 - val_loss: 0.9020
Epoch 2/10
1133/1133 [==============================] - 3s 3ms/step - loss: 0.8865 - val_loss: 0.8881
Epoch 3/10
1133/1133 [==============================] - 3s 2ms/step - loss: 0.8722 - val_loss: 0.8836
Epoch 4/10
1133/1133 [==============================] - 3s 2ms/step - loss: 0.8614 - val_loss: 0.8828
Epoch 5/10
1133/1133 [==============================] - 3s 2ms/step - loss: 0.8464 - val_loss: 0.8668
Epoch 6/10
1133/1133 [==============================] - 4s 4ms/step - loss: 0.8328 - val_loss: 0.8613
Epoch 7/10
1133/1133 [==============================] - 4s 3ms/step - loss: 0.8222 - val_loss: 0.8592
Epoch 8/10
1133/1133 [==============================] - 3s 3ms/step - loss: 0.8115 - val_loss: 0.8571
Epoch 9/10
1133/1133 [==============================] - 3s 3ms/step - loss: 0.7994 - val_loss: 0.8560
Epoch 10/10
567/567 [==============================] - 1s 2ms/step - loss: 0.8603